<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/deneme_untitled_0_best_model_2_from_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Libraries and Indicator Function

!pip install -q bta-lib
!pip install -q ta

import btalib
import numpy as np
import pandas as pd
from ta.trend import PSARIndicator
from ta.momentum import WilliamsRIndicator
from ta.trend import AroonIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.trend import CCIIndicator
from ta.momentum import ROCIndicator
from ta.trend import ADXIndicator
import yfinance as yf
import warnings
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

warnings.filterwarnings("ignore")


def create_indicators(data):
  data["RSI"] = btalib.rsi(data["Close"], period=14).df
  data["SMA5"] = btalib.sma(data['Close'], period=5).df
  data["SMA9"] = btalib.sma(data['Close'], period=9).df
  data["SMA10"] = btalib.sma(data['Close'], period=10).df
  data["SMA14"] = btalib.sma(data['Close'], period=14).df
  data["SMA20"] = btalib.sma(data['Close'], period=20).df
  data["SMA21"] = btalib.sma(data['Close'], period=21).df
  data["SMA50"] = btalib.sma(data['Close'], period=50).df
  data["SMA200"] = btalib.sma(data['Close'], period=200).df
  data["EMA5"] = btalib.ema(data['Close'], period=5).df
  data["EMA14"] = btalib.ema(data['Close'], period=14).df
  data["EMA21"] = btalib.ema(data['Close'], period=21).df
  data["EMA50"] = btalib.ema(data['Close'], period=50).df
  data["STOCH-K"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["k"]
  data["STOCH-D"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["d"]
  data["MACD"] = btalib.macd(data['Close']).df["macd"]
  data["SIGNAL"] = btalib.macd(data['Close']).df["signal"]
  data["HISTOGRAM"] = btalib.macd(data['Close']).df["histogram"]
  data["BB-UPPER"] = btalib.bbands(data['Close']).df['top']
  data["BB-MID"] = btalib.bbands(data['Close']).df['mid']
  data["BB-LOWER"] = btalib.bbands(data['Close']).df['bot']
  data["STDEV"] = data["Close"].rolling(window=10).std()
  data["PSAR"] = PSARIndicator(data["High"], data["Low"], data["Close"]).psar()
  data["WILLIAMS"] = WilliamsRIndicator(data["High"], data["Low"], data["Close"]).williams_r()
  data["AROON"] = AroonIndicator(close=data["Close"], window=25).aroon_indicator()
  data["OBV"] = VolumePriceTrendIndicator(close=data['Close'], volume=data['Volume']).volume_price_trend()
  data['CCI'] = CCIIndicator(close=data['Close'], low=data["Low"], high=data["High"], window=14).cci()
  data['ROC'] = ROCIndicator(close=data['Close'], window=5).roc()
  data['BULL'] = data['High'] - (data['High'].rolling(13).max() + data['Low'].rolling(13).min()) / 2
  data['BEAR'] = data['Low'] - (data['High'].rolling(13).max() + data['Low'].rolling(13).min()) / 2
  adx_indicator = ADXIndicator(high=data['High'], low=data['Low'], close=data['Close'], window=14)
  data['ADX'] = adx_indicator.adx()
  data['+DI'] = adx_indicator.adx_pos()
  data['-DI'] = adx_indicator.adx_neg()

  data = data.dropna()
  data = data.reset_index()
  return data

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Mounted at /content/drive


In [23]:
#@title Download the Model

import gdown

# gdown.download("https://drive.google.com/u/1/uc?id=1qWKIj9rzRzyW3GQFw9xISs4eXmjyIDng&export=download", "/content/", quiet=False)
# gdown.download("https://drive.google.com/u/0/uc?id=117pezAA6jRLCwIsdpEhZgEa9tEanlC0O&export=download", "/content/", quiet=False)

data = pd.read_csv("data.csv")
model = tf.keras.models.load_model("best_model.h5")

In [28]:
#@title Stock List
sheet_id = "1RSqOXkFTAO7g4H9LEY3d3IX6H6bJaYk1"
sheet_name = "Sheet_1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
result_df = pd.read_csv(url)

sheet_id = "1AA9MfqOtAAgO97__aomD79DciyT-PkRQ"
sheet_name = "Sheet_1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
result_df = pd.read_csv(url)

nasdaq100 = ['AAPL', 'MSFT', 'GOOGL', 'GOOG', 'AMZN',
             'NVDA', 'TSLA', 'META', 'AVGO', 'ASML',
             'PEP', 'COST', 'ADBE', 'AZN', 'CSCO',
             'NFLX', 'AMD', 'CMCSA', 'TMUS', 'TXN',
             'QCOM', 'HON', 'INTU', 'INTC', 'SNY',
             'VZ', 'AMGN', 'SBUX', 'ISRG', 'AMAT',
             'BKNG', 'ADI', 'MDLZ', 'PDD', 'GILD',
             'ADP', 'VRTX', 'ABNB', 'LRCX', 'PYPL',
             'REGN', 'EQIX', 'MU', 'CSX', 'SNPS',
             'CME', 'CDNS', 'KLAC', 'NTES']

In [30]:
data = pd.DataFrame()

for stock in result_df["STOCK"].unique():
  stock_df = yf.download(stock, start="2000-01-01", end="2023-01-01", progress=False)
  stock_df = create_indicators(stock_df)
  stock_df["signal"] = 0

  for index, row in stock_df.iterrows():
    if index > 0 and index < stock_df.shape[0]-1 and stock_df["Close"].iloc[index+1] > ((2.5*stock_df["Close"].iloc[index]/100) + stock_df["Close"].iloc[index]):
      stock_df["signal"].iloc[index] = 1

  stock_df = stock_df.drop(["Date", "Open", "High", "Low", "Close", "Volume", "Adj Close"], axis=1)

  data = pd.concat([data, stock_df], ignore_index=True)

In [31]:
#@title Data Preprocessing
df = pd.DataFrame(data)

input_columns = df.columns[:33]
output_column = "signal"

df[output_column] = df[output_column].astype(int)

X = df[input_columns].values
y = df[output_column].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight("balanced", classes=[0, 1], y=y_train)
class_weight = {cls: weight for cls, weight in zip([0, 1], class_weights)}
class_weight

{0: 0.5752297447063093, 1: 3.823153640571019}

In [ ]:
#@title LSTM Model

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_resampled.shape[1], 1)),

    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.LSTM(256),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=256, validation_split=0.2, callbacks=[early_stopping, model_checkpoint])

In [ ]:
#@title Conv1D model
from tensorflow import keras
from tensorflow.keras import layers

X_train_resampled = np.expand_dims(X_train_resampled, axis=2)
X_test = np.expand_dims(X_test, axis=2)

def create_model():
    input_layer = keras.Input(shape=(X_train_resampled.shape[1], X_train_resampled.shape[2]))

    x = layers.Conv1D(
        filters=32, kernel_size=3, strides=2, activation="relu", padding="same"
    )(input_layer)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(
        filters=64, kernel_size=3, strides=2, activation="relu", padding="same"
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(
        filters=128, kernel_size=5, strides=2, activation="relu", padding="same"
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(
        filters=256, kernel_size=5, strides=2, activation="relu", padding="same"
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(
        filters=512, kernel_size=7, strides=2, activation="relu", padding="same"
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(
        filters=1024, kernel_size=7, strides=2, activation="relu", padding="same"
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dropout(0.2)(x)

    x = layers.Flatten()(x)

    x = layers.Dense(4096, activation="relu")(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(
        2048, activation="relu", kernel_regularizer=keras.regularizers.L2()
    )(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(
        1024, activation="relu", kernel_regularizer=keras.regularizers.L2()
    )(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(
        128, activation="relu", kernel_regularizer=keras.regularizers.L2()
    )(x)
    output_layer = layers.Dense(1, activation="sigmoid")(x)

    return tf.keras.Model(inputs=input_layer, outputs=output_layer)

model = create_model()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/best_model_2.h5', save_best_only=False)

model.fit(X_train_resampled, y_train_resampled, epochs=200, batch_size=512, validation_split=0.2,
          callbacks=[early_stopping, model_checkpoint], class_weight=class_weight)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/200
319/319 [==============================] - 38s 94ms/step - loss: 3.6090 - accuracy: 0.3753 - val_loss: 0.3291 - val_accuracy: 1.0000
Epoch 2/200
319/319 [==============================] - 34s 105ms/step - loss: 0.8959 - accuracy: 0.3750 - val_loss: 0.2382 - val_accuracy: 1.0000
Epoch 3/200
319/319 [==============================] - 38s 120ms/step - loss: 0.8761 - accuracy: 0.3750 - val_loss: 0.2497 - val_accuracy: 1.0000
Epoch 4/200
319/319 [==============================] - 33s 105ms/step - loss: 0.8679 - accuracy: 0.3750 - val_loss: 0.2611 - val_accuracy: 1.0000
Epoch 5/200
319/319 [==============================] - 34s 108ms/step - loss: 0.8618 - accuracy: 0.3750 - val_loss: 0.1930 - val_accuracy: 1.0000
Epoch 6/200
319/319 [==============================] - 33s 102ms/step - loss: 0.8547 - accuracy: 0.3750 - val_loss: 0.2198 - val_accuracy: 1.0000
Epoch 7/200
319/319 [==============================] - 39s 122ms/step - loss: 0.8465 - accuracy: 0.3750 - val_loss: 0.2081 - 

In [ ]:
# model.save("xu030-long-deneme.h5")
# model = tf.keras.models.load_model("best_model.h5")
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
for stock in ["TUPRS.IS"]:
  predictions = []

  stock_data = yf.download(stock, start="2000-01-01", end="2023-08-10", progress=False)
  stock_data = create_indicators(stock_data)

  stock_data["signal"] = 0

  for row in range(stock_data.shape[0]):
    if row+1 != stock_data.shape[0] and stock_data["Close"].iloc[row+1] > stock_data["Close"].iloc[row] + 5*(stock_data["Close"].iloc[row])/100:
      stock_data["signal"].iloc[row] = 1

  stock_data = stock_data.drop(["Open", "High", "Low", "Close", "Adj Close", "Volume"], axis=1)

  for index, row in stock_data.iterrows():
      x = row[1:34]

      new_data = x.to_numpy().reshape(1, -1)
      new_data = scaler.transform(new_data)
      prediction = model.predict(new_data, verbose=None)

      # print(f"Prediction for date {row[0]}: {np.round(prediction[0][0])}")

      if np.round(prediction[0][0]) == row["signal"]:
        predictions.append(1)
      elif np.round(prediction[0][0]) != row["signal"]:
        predictions.append(0)

  print(f"{stock} Accuracy: {predictions.count(1) / len(predictions) * 100}")

In [18]:
buy_stocks = []

for stock in result_df["STOCK"].unique():
  stock_data = yf.download(stock, start="2021-06-01", end="2023-08-16", progress=False)
  stock_data = create_indicators(stock_data)

  change = ((stock_data["Close"].iloc[-1] - stock_data["Close"].iloc[-2]) / stock_data["Close"].iloc[-2])*100
  change = round(change, 2)

  stock_data = stock_data.drop(["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"], axis=1)

  x = stock_data.iloc[-2]

  new_data = x.to_numpy().reshape(1, -1)
  new_data = scaler.transform(new_data)
  prediction = model.predict(new_data, verbose=None)

  if round(prediction[0][0]*100,2) > 75:
    buy_stocks.append([stock, round(prediction[0][0]*100,2), change])

buy_df = pd.DataFrame(buy_stocks, columns=["stock", "probability", "change"]).sort_values(by="probability", ascending=False)
print(f'Pozitif kapanan hisse sayısı: {buy_df[buy_df["change"] > 0].shape[0]}')
print(f'Negatif kapanan hisse sayısı: {buy_df[buy_df["change"] < 0].shape[0]}')
print(f'Günlük değişim ortalaması: %{round(buy_df["change"].sum()/buy_df.shape[0], 2)}\n')
buy_df

Pozitif kapanan hisse sayısı: 9
Negatif kapanan hisse sayısı: 14
Günlük değişim ortalaması: %-0.53



,stock,probability,change
5,ALKA.IS,99.95,-1.79
1,TAVHL.IS,99.70,1.33
3,ADESE.IS,99.59,-9.76
15,GLCVY.IS,99.01,0.00
2,ADEL.IS,97.87,9.99
8,BRSAN.IS,97.72,-2.29
6,ANGEN.IS,97.22,-2.04
16,NETAS.IS,96.71,-4.69
18,POLTK.IS,96.11,0.00
11,ERSU.IS,95.81,-8.20
